In [2]:
!pip install scikit-learn

  Using cached scikit_learn-1.3.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.11.4-cp311-cp311-macosx_12_0_arm64.whl.metadata (165 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
Using cached scikit_learn-1.3.2-cp311-cp311-macosx_12_0_arm64.whl (9.4 MB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached scipy-1.11.4-cp311-cp311-macosx_12_0_arm64.whl (29.7 MB)
Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import sqlite3
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# データベースへの接続
conn = sqlite3.connect('tenki.sqlite')
cursor = conn.cursor()

# データベースからデータの取得
cursor.execute('SELECT date, pressure, temperature, humidity FROM weather_data')
data1 = cursor.fetchall()

cursor.execute('SELECT date, steps, stride_median, calories_burned, sedentary_calories_burned FROM local_tenki')
data2 = cursor.fetchall()

# データベース接続を閉じる
conn.close()

# データをNumPyの配列に変換
data1_array = np.array(data1)
data2_array = np.array(data2)

# コサイン類似度の計算
cosine_similarity_matrix = cosine_similarity(data1_array, data2_array)
cosine_similarity_value = cosine_similarity_matrix[0, 0]  # 例として1つの類似度を取得

print("コサイン類似度:", cosine_similarity_value)

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 4 while Y.shape[1] == 5